### Imports

In [1]:
from datetime import datetime
import time

from contracts_lib_py.account import Account
from common_utils_py.agreements.service_types import ServiceTypesIndices


from nevermined_sdk_py import Config, Nevermined
from nevermined_sdk_py.nevermined.keeper import NeverminedKeeper as Keeper

import utils

CONSUMER_ADDRESS = "0x00Bd138aBD70e2F00903268F3Db08f2D25677C9e"
CONSUMER_PASSWORD = "node0"
CONSUMER_KEYFILE = "../resources/accounts/consumer.json"
CONFIG_FILE = "../config.ini"

COORDINATOR_URL = "http://172.17.0.2:8080"

PROVIDER_ADDRESS = "0x068Ed00cF0441e4829D9784fCBe7b9e26D4BD8d0"
ASSET_COMPUTE_DID_1 = "did:nv:8b4e3316a87ac4b9288280477c440f790292ece1fc90b172888730514b250218"
ASSET_COMPUTE_DID_2 = "did:nv:53eb725fae3d3ac19338a59587489990c523c237fdc5cb4f2e7f6f99bcb9aecb"
ASSET_FLOWER_SERVER_DID = "did:nv:80dee7612bed0abc5cc83f1c9d7a479c27eb02aa5a838a91940244d93d266121"

### Setup Nevermined and accounts

In [2]:
nevermined = Nevermined(Config(CONFIG_FILE))
consumer_account = Account(CONSUMER_ADDRESS, CONSUMER_PASSWORD, CONSUMER_KEYFILE)

nevermined.accounts.request_tokens(consumer_account, 10)


True

### Publish algorithm

In [3]:
metadata_algorithm = utils.metadata_algorithm_template()

metadata_algorithm["main"]["dateCreated"] = utils.date_now()
metadata_algorithm["main"]["files"][0]["url"] = "https://github.com/nevermined-io/fl-demo/raw/feature/flower/flower/client.py"

metadata_algorithm["main"]["algorithm"]["entrypoint"] = (
    "pip install tensorflow-cpu flwr==0.15 && "
    "python client.py 172.17.0.2:8080"
)

metadata_algorithm["main"]["algorithm"]["requirements"]["container"]["image"] = "python"
metadata_algorithm["main"]["algorithm"]["requirements"]["container"]["tag"] = "3.8-slim-buster"

utils.print_json(metadata_algorithm)

ddo_algorithm = nevermined.assets.create(metadata_algorithm, consumer_account, providers=[PROVIDER_ADDRESS])
print(f"Algorithm DID: {ddo_algorithm.did}")

{
  "main": {
    "name": "Fraud Demo Algorithm",
    "dateCreated": "2021-03-15T13:02:46Z",
    "author": "Nevermined Consumer",
    "license": "",
    "price": "0",
    "files": [
      {
        "index": 0,
        "contentType": "text/text",
        "checksum": "0x52b5c93b82dd9e7ecc3d9fdf4755f7f69a54484941897dc517b4adfe3bbc3377",
        "checksumType": "MD5",
        "contentLength": "12057507",
        "url": "https://github.com/nevermined-io/fl-demo/raw/feature/flower/flower/client.py"
      }
    ],
    "type": "algorithm",
    "algorithm": {
      "language": "python",
      "format": "py",
      "version": "0.1.0",
      "entrypoint": "pip install tensorflow-cpu flwr==0.15 && python client.py 172.17.0.2:8080",
      "requirements": {
        "container": {
          "image": "python",
          "tag": "3.8-slim-buster",
          "checksum": "sha256:53ad3a03b2fb240b6c494339821e6638cd44c989bcf26ec4d51a6a52f7518c1d"
        }
      }
    }
  }
}
Algorithm DID: did:nv:6f67dea84c

### Publish the workflows:

- Two for the compute to the data assets
- One for the coordinator service

In [4]:
# Compute asset 1
metadata_workflow1 = utils.metadata_workflow_template()

metadata_workflow1["main"]["dateCreated"] = utils.date_now()
metadata_workflow1["main"]["workflow"]["stages"][0]["input"] = []
metadata_workflow1["main"]["workflow"]["stages"][0]["transformation"]["id"] = ddo_algorithm.did

utils.print_json(metadata_workflow1)

{
  "main": {
    "name": "Fraud Demo Workflow",
    "dateCreated": "2021-03-15T13:02:48Z",
    "author": "Nevermined Consumer",
    "license": "",
    "price": "0",
    "type": "workflow",
    "workflow": {
      "stages": [
        {
          "index": 0,
          "input": [],
          "transformation": {
            "id": "did:nv:6f67dea84c707c2658449c47afa3123d0a0c2326eacdeb7fdafc2740038a5be8"
          }
        }
      ]
    }
  }
}


In [5]:
ddo_workflow1 = nevermined.assets.create(metadata_workflow1, consumer_account, providers=[PROVIDER_ADDRESS])
print(f"Workflow DID: {ddo_workflow1.did}")

Workflow DID: did:nv:d9e72e4a36c08335cb8c613b840b4168d725aeca5476434c33668483b5d7bbe2


In [6]:
# Compute asset 2
metadata_workflow2 = utils.metadata_workflow_template()
metadata_workflow2["main"]["dateCreated"] = utils.date_now()
metadata_workflow2["main"]["workflow"]["stages"][0]["input"] = []
metadata_workflow2["main"]["workflow"]["stages"][0]["transformation"]["id"] = ddo_algorithm.did

utils.print_json(metadata_workflow2)

ddo_workflow2 = nevermined.assets.create(metadata_workflow2, consumer_account, providers=[PROVIDER_ADDRESS])
print(f"Workflow DID: {ddo_workflow2.did}")

{
  "main": {
    "name": "Fraud Demo Workflow",
    "dateCreated": "2021-03-15T13:02:51Z",
    "author": "Nevermined Consumer",
    "license": "",
    "price": "0",
    "type": "workflow",
    "workflow": {
      "stages": [
        {
          "index": 0,
          "input": [],
          "transformation": {
            "id": "did:nv:6f67dea84c707c2658449c47afa3123d0a0c2326eacdeb7fdafc2740038a5be8"
          }
        }
      ]
    }
  }
}
Workflow DID: did:nv:0171138f60be10ce974c89a607d8b392bb8ad6e0d3bd614e733c3198113e2fa0


In [7]:
# Compute server
metadata_workflow3 = utils.metadata_workflow_template()
metadata_workflow3["main"]["dateCreated"] = utils.date_now()
metadata_workflow3["main"]["workflow"]["stages"][0]["input"] = []
metadata_workflow3["main"]["workflow"]["stages"][0]["transformation"]["id"] = ASSET_FLOWER_SERVER_DID

utils.print_json(metadata_workflow3)

ddo_workflow3 = nevermined.assets.create(metadata_workflow3, consumer_account, providers=[PROVIDER_ADDRESS])
print(f"Workflow DID: {ddo_workflow3.did}")

{
  "main": {
    "name": "Fraud Demo Workflow",
    "dateCreated": "2021-03-15T13:02:54Z",
    "author": "Nevermined Consumer",
    "license": "",
    "price": "0",
    "type": "workflow",
    "workflow": {
      "stages": [
        {
          "index": 0,
          "input": [],
          "transformation": {
            "id": "did:nv:80dee7612bed0abc5cc83f1c9d7a479c27eb02aa5a838a91940244d93d266121"
          }
        }
      ]
    }
  }
}
Workflow DID: did:nv:882842f1b55e6c291298a6ffeaedfeb0fc0afb811dcf46bc1f798aeee5a8ae5f


### Order computations:

- One for each data asset
- One for the coordinator service

In between we want for the blockchain to process the request.

In [8]:
keeper = Keeper.get_instance()

service_agreement_id1 = nevermined.assets.order(ASSET_COMPUTE_DID_1, ServiceTypesIndices.DEFAULT_COMPUTING_INDEX, consumer_account, consumer_account)
print(f"Service Agreement ID: {service_agreement_id1}")
utils.wait_for_event(keeper, service_agreement_id1)

service_agreement_id2 = nevermined.assets.order(ASSET_COMPUTE_DID_2, ServiceTypesIndices.DEFAULT_COMPUTING_INDEX, consumer_account, consumer_account)
print(f"Service Agreement ID: {service_agreement_id2}")
utils.wait_for_event(keeper, service_agreement_id2)

#service_agreement_id_coordinator = nevermined.assets.order(ASSET_FLOWER_SERVER_DID, ServiceTypesIndices.DEFAULT_COMPUTING_INDEX, consumer_account, consumer_account)
#print(f"Service Agreement ID: {service_agreement_id_coordinator}")
#utils.wait_for_event(keeper, service_agreement_id_coordinator)


Service Agreement ID: 0x58103d6ed3b74b55943bbd6c9c3cc54c34aa385b372c44b8829727d3610d70ee
Service Agreement ID: 0x66ca430311db4b6fbd3b52fb0139669366b8af43ecc842f0943b60adea7b6f85


### Execute workflows

This orders the Nevermined to start the execution of the workflows:
- One for each data asset
- One for the coordinator service

In [9]:
execution_id1 = nevermined.assets.execute(
    service_agreement_id1,
    ASSET_COMPUTE_DID_1,
    ServiceTypesIndices.DEFAULT_COMPUTING_INDEX,
    consumer_account,
    ddo_workflow1.did
)
print(f"Execution ID: {execution_id1}")

Execution ID: nevermined-compute-2pxfx


In [10]:
execution_id2 = nevermined.assets.execute(
    service_agreement_id2,
    ASSET_COMPUTE_DID_2,
    ServiceTypesIndices.DEFAULT_COMPUTING_INDEX,
    consumer_account,
    ddo_workflow2.did
)
print(f"Execution ID: {execution_id2}")

Execution ID: nevermined-compute-mxm4g


In [11]:
execution_id_coordinator = nevermined.assets.execute(
    service_agreement_id2,
    ASSET_COMPUTE_DID_2,
    ServiceTypesIndices.DEFAULT_COMPUTING_INDEX,
    consumer_account,
    ddo_workflow3.did
)
print(f"Execution ID: {execution_id_coordinator}")

Execution ID: nevermined-compute-scsdn


### Wait for compute jobs to finish

In [12]:
succeeded = set()
failed = set()
outputs = set()
while len(succeeded) < 3:
    for job_id in [execution_id1, execution_id2, execution_id_coordinator]:
        result = nevermined.assets.compute_status(service_agreement_id1, job_id, consumer_account)
        status = result["status"]
        print(f"{job_id}: {status}")

        if status == "Succeeded":
            succeeded.add(job_id)
            outputs.add(result["did"])
        elif status == "Failed":
            raise Exception("Some jobs failed")
    print()
    time.sleep(10)

for did in outputs:
    print(f"Output DID: {did}")

nevermined-compute-2pxfx: Running
nevermined-compute-mxm4g: Running
nevermined-compute-scsdn: Running

nevermined-compute-2pxfx: Running
nevermined-compute-mxm4g: Running
nevermined-compute-scsdn: Running

nevermined-compute-2pxfx: Running
nevermined-compute-mxm4g: Running
nevermined-compute-scsdn: Running

nevermined-compute-2pxfx: Running
nevermined-compute-mxm4g: Running
nevermined-compute-scsdn: Running

nevermined-compute-2pxfx: Running
nevermined-compute-mxm4g: Running
nevermined-compute-scsdn: Running

nevermined-compute-2pxfx: Running
nevermined-compute-mxm4g: Running
nevermined-compute-scsdn: Running

nevermined-compute-2pxfx: Running
nevermined-compute-mxm4g: Running
nevermined-compute-scsdn: Running

nevermined-compute-2pxfx: Running
nevermined-compute-mxm4g: Running
nevermined-compute-scsdn: Running

nevermined-compute-2pxfx: Running
nevermined-compute-mxm4g: Running
nevermined-compute-scsdn: Running

nevermined-compute-2pxfx: Running
nevermined-compute-mxm4g: Running
never

Exception: Some jobs failed

### Download the results

In [ ]:
for did in outputs:
    print(f"Downloading: {did}")
    nevermined.assets.download(did, ServiceTypesIndices.DEFAULT_ACCESS_INDEX, consumer_account, "./")

In [ ]:
nevermined.assets.compute_status(service_agreement_id1, job_id, consumer_account)